importing and merging LTL and **warehouse**

In [28]:
import pandas as pd

# Load the CSV files
data = pd.read_csv("Wheel Pros Baseline LTL Data 2023 (1).csv")
df = pd.read_csv("/content/Warehouse Listing and Shipping Acct Numbers 23-0927.csv")

# Rename the column in df to match the one in data
df.rename(columns={'Zip': 'Shipper Zip'}, inplace=True)

data = data[~data['Recipient State/Province'].isin(['SK', 'QC', 'ON', 'AB', 'BC', 'NB', 'NS', 'NL', 'PE'])]

num_rows = data.shape[0]
print("Number of rows in the DataFrame:", num_rows)

# Merge based on the 'Shipper Zip' column
result_df = pd.merge(data, df[['Shipper Zip', 'Location']], on='Shipper Zip', how='left')
print(result_df)


# Display the first few rows of the dataset
#print(type(data[]))


Number of rows in the DataFrame: 26640
             Carrier    Account Tracking Number         BOL Number PO Number  \
0       R+L Carriers     WHE979       817902783         1014484360       NaN   
1       R+L Carriers     WHE534       508568274  NONE                    NaN   
2       R+L Carriers     WHE534       508550283  NONE                    NaN   
3       R+L Carriers     WHE534        75825603  NONE                    NaN   
4       R+L Carriers     WHE702       718865631  NONE                    NaN   
...              ...        ...             ...                ...       ...   
47644   R+L Carriers     WHE534       310433202  NONE                    NaN   
47645   R+L Carriers     W15824       941098644  800-337-9005            NaN   
47646   R+L Carriers     W15824       177191991  608-290-3396            NaN   
47647  FedEx Freight  704813392      5033052791                NaN      NOPO   
47648   R+L Carriers     W15824       195324039  RMA032397               NaN   



**formatting LTL merge**

In [29]:
# Keep only the selected columns
selected_columns = ["Carrier", "Shipment Date", "Shipper City", "Shipper Zip","Shipper State", "Location",
                    "Recipient City", "Recipient Zip", "Recipient State/Province", "Weight (lbs)", "Net Charge", "Miles"]
result_df = result_df[selected_columns]

result_df['Weight (lbs)'] = result_df['Weight (lbs)'].str.replace(',','').astype(float)
result_df['Net Charge'] = result_df['Net Charge'].str.replace('$','').str.replace(',','').astype(float)
result_df['Miles'] = result_df['Miles'].str.replace(',','').astype(float)

# numeric_columns = ["Weight (lbs)", "Miles"]
# data[numeric_columns] = data[numeric_columns].apply(pd.to_numeric, errors='coerce')
# Display the first few rows of the updated dataset
print(result_df[['Weight (lbs)']].head())
print(result_df[['Net Charge']].head())
print(result_df[['Miles']].head())

result_df.to_csv("mother_source.csv", index=False)


<ipython-input-29-f943417f92f2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['Weight (lbs)'] = result_df['Weight (lbs)'].str.replace(',','').astype(float)
<ipython-input-29-f943417f92f2>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  result_df['Net Charge'] = result_df['Net Charge'].str.replace('$','').str.replace(',','').astype(float)
<ipython-input-29-f943417f92f2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

   Weight (lbs)
0        5000.0
1        3925.0
2        2424.0
3        2166.0
4        1491.0
   Net Charge
0      228.61
1      959.97
2      601.46
3      434.86
4      277.18
    Miles
0   113.0
1  1324.0
2  1324.0
3   529.0
4   411.0


**LTL aggregation @ demand end**

In [30]:

# Convert 'Shipment Date' to datetime format
result_df['Shipment Date'] = pd.to_datetime(result_df['Shipment Date'], format='%d-%m-%Y')
# Extract month from 'Shipment Date'
result_df['Month'] = result_df['Shipment Date'].dt.month
#print(data['Month'].unique())
result_df['Recipient City'] = result_df['Recipient City'].str.replace(' ', '').str.title()

# Aggregate at zip level
zip_aggregated = result_df.groupby(['Recipient Zip', 'Month', 'Recipient City', 'Recipient State/Province']).agg({
    'Weight (lbs)': 'sum',
    'Net Charge': 'sum'
}).reset_index()

zip_aggregated['Country'] = 'USA'
zip_aggregated['type'] = 'LTL'
# Display aggregated data at zip level

print(zip_aggregated.head())
zip_aggregated.to_csv("demand_aggregated_final.csv", index=False)




  Recipient Zip  Month Recipient City Recipient State/Province  Weight (lbs)  \
0    02766-2913      3         Norton                       MA         584.0   
1    06042-2211      3     Manchester                       CT        3108.0   
2    06042-2211      4     Manchester                       CT        9162.0   
3    06042-2211      9     Manchester                       CT        2044.0   
4    06051-2915      8     Newbritain                       CT         144.0   

   Net Charge Country type  
0      417.78     USA  LTL  
1     2331.52     USA  LTL  
2     2398.41     USA  LTL  
3      517.80     USA  LTL  
4      182.51     USA  LTL  


**LTL aggregation @ supplier end**

In [31]:
# Convert 'Shipment Date' to datetime format
result_df['Shipment Date'] = pd.to_datetime(result_df['Shipment Date'], format='%d-%m-%Y')
# Extract month from 'Shipment Date'
result_df['Month'] = result_df['Shipment Date'].dt.month
#print(data['Month'].unique())
result_df['Shipper City'] = result_df['Shipper City'].str.replace(' ', '').str.title()

# Aggregate at zip level
zip_aggregated = result_df.groupby(['Shipper Zip', 'Month', 'Shipper City', 'Shipper State', 'Location']).agg({
    'Weight (lbs)': 'sum',
    'Net Charge': 'sum'
}).reset_index()

zip_aggregated['Country'] = 'USA'
zip_aggregated['type'] = 'LTL'
# Display aggregated data at zip level

print(zip_aggregated.head())
zip_aggregated.to_csv("supply_aggregated.csv", index=False)

  Shipper Zip  Month Shipper City Shipper State Location  Weight (lbs)  \
0       11735      1  Farmingdale            NY     1042       12530.0   
1       11735      2  Farmingdale            NY     1042       19405.0   
2       11735      3  Farmingdale            NY     1042       13842.0   
3       11735      4  Farmingdale            NY     1042       36269.0   
4       11735      5  Farmingdale            NY     1042        9589.0   

   Net Charge Country type  
0     5386.86     USA  LTL  
1     8218.38     USA  LTL  
2     6543.49     USA  LTL  
3    21401.75     USA  LTL  
4     5329.54     USA  LTL  


In [ ]:
#print(data[data['Month']==12])

**FTL WORKING**

In [33]:
data1 = pd.read_csv("/content/FTL Baseline Jan - Oct 2023.csv")
selected_columns = ["Carrier Name","O: Pick Up Name", "Ship Date","O: City", "O: Zip", "O: State",
                    "D: City", "D: Zip", "D: State", "Weight", "Total Charges"]
data1 = data1[data1['O: Zip'].isin(['90620','75050','30519'])]
data1 = data1[selected_columns]
data1['Weight'] = data1['Weight'].str.replace(',','').astype(float)
data1['Total Charges'] = data1['Total Charges'].str.replace('$','').str.replace(',','').astype(float)
print(data1[['Weight']].head())
print(data1[['Total Charges']].head())

num_rows = data1.shape[0]
print("Number of rows in the DataFrame:", num_rows)


df = pd.read_csv("/content/Warehouse Listing and Shipping Acct Numbers 23-0927.csv")

# Rename the column in df to match the one in data
df.rename(columns={'Zip': 'D: Zip'}, inplace=True)

result_df = pd.merge(data1, df[['D: Zip', 'Location']], on='D: Zip', how='left')

print(result_df)
result_df.to_csv("FTL load.csv", index=False)



     Weight
0   35000.0
6   35000.0
7   35000.0
9   35000.0
12  35000.0
    Total Charges
0         1960.00
6         2191.05
7         2178.00
9         1125.00
12        3700.00
Number of rows in the DataFrame: 2473
     Carrier Name   O: Pick Up Name   Ship Date        O: City O: Zip  \
0        Landstar  Wheel Pros #1086  01-03-2023  Grand Prairie  75050   
1        Landstar  Wheel Pros #1086  01-03-2023  Grand Prairie  75050   
2         Connect  Wheel Pros #1088  01-03-2023         Buford  30519   
3     CH Robinson  Wheel Pros #1086  01-03-2023  Grand Prairie  75050   
4         Connect  Wheel Pros #1088  01-03-2023         Buford  30519   
...           ...               ...         ...            ...    ...   
3106      Connect  Wheel Pros #1088    10-31-23         Buford  30519   
3107       Celtic  Wheel Pros #1088    10-31-23         Buford  30519   
3108      Connect  Wheel Pros #1085    10-31-23     Buena Park  90620   
3109      Connect  Wheel Pros #1088    10-31-23     

<ipython-input-33-3f9a5b5ebc14>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data1['Total Charges'] = data1['Total Charges'].str.replace('$','').str.replace(',','').astype(float)


       Carrier  Month  Weight (lbs)  Net Charge
0  CH Robinson      1       96082.0    36099.32
1  CH Robinson      2       86128.0    27836.95
2  CH Robinson      3      126576.0    41686.41
3  CH Robinson      4       57310.0    18530.97
4  CH Robinson      5      153828.0    62473.26


In [ ]:
#zip_aggregated.to_csv("zip_aggregated_final.csv", index=False)
#city_aggregated.to_csv("city_aggregated_final.csv", index=False)
#carrier_aggregated.to_csv("carrier_aggregated_final.csv", index=False)